<a href="https://colab.research.google.com/github/columbia-data-club/meetings/blob/main/2023/April_13_Xarrays_Children.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <img src="https://i.etsystatic.com/7750602/r/il/e14fba/2242302522/il_570xN.2242302522_fbes.jpg" alt="Drawing" style="width: 450px;"/> </center>

## Xarray's Children

April 13th, 2023

by [Roger Creel](https://rogercreel.com) for the [Columbia Data Club](https://github.com/columbia-data-club/).

This notebook underpins a ~60-75 minute presentation that discusses several python libraries built on top of Xarray.  The notebook draws heavily on the documentation of each of these libraries. It is geared towards beginners to Python. 


# **Xarray's Children**
*Presented by Columbia University Libraries*
***

Welcome to the Columbia University Library's workshop on the python library offspring of Xarray! These are our objectives:


* Review Xarray as a tool for data analysis
* Introduce MetPy, a Python library for meteorogical observations
* Mention Pint, a python library for unit management



* Introduce OGGM, a Python library for glacier modeling
* Mention Salem, a pythoon library for data processing and plotting



* Answer questions! 






## Xarray primer
You may be familiar with [Pandas](https://pandas.pydata.org/pandas-docs/stable/) or [Geopandas](http://geopandas.org) as excellent libraries for analyzing tabular "labeled data". [Xarray](http://xarray.pydata.org/en/stable/) is designed to make it easier to work with with _labeled multidimensional data_. By _multidimensional data_ (also often called _N-dimensional_), we mean data with many independent dimensions or axes. For example, we might represent Earth's surface temperature $T$ as a three dimensional variable

$$ T(x, y, t) $$

where $x$ and $y$ are spatial dimensions and and $t$ is time. 

By _labeled_, we mean data that has metadata associated with it describing the names and relationships between the variables. The cartoon below shows a "data cube" schematic dataset with temperature and preciptation sharing the same three dimensions, plus longitude and latitude as auxilliary coordinates.


The point of xarray is to provide pandas-level convenience for working with this type of data.


![xarray data model](https://github.com/pydata/xarray/blob/main/doc/_static/dataset-diagram.png?raw=true)

### Xarray data structures

Like Pandas, xarray has two fundamental data structures:
* a `DataArray`, which holds a single multi-dimensional variable and its coordinates
* a `Dataset`, which holds multiple variables that potentially share the same coordinates

#### DataArray

A `DataArray` has four essential attributes:
* `values`: a `numpy.ndarray` holding the array’s values
* `dims`: dimension names for each axis (e.g., `('x', 'y', 'z')`)
* `coords`: a dict-like container of arrays (coordinates) that label each point (e.g., 1-dimensional arrays of numbers, datetime objects or strings)
* `attrs`: an `OrderedDict` to hold arbitrary metadata (attributes)

#### DataSet

A dataset is simply an object containing multiple DataArrays indexed by variable name




----------------------------
<center> <img src="https://unidata.github.io/MetPy/latest/_static/metpy_horizontal.png" alt="Drawing" style="width: 450px;"/>


# MetPy: meteorological calculations with xarray 

### (With cameo appearance by Pint)



<td> <img src="https://pint.readthedocs.io/en/stable/_static/logo-full.jpg" alt="Drawing" style="width: 450px;"/> </td>

 </center>

 MetPy's suite of meteorological calculations are designed to integrate with xarray DataArrays as one of its two
primary data models (the other being Pint Quantities). MetPy also provides DataArray and
Dataset *accessors* (collections of methods and properties attached to the ``.metpy`` property)
for coordinate/CRS and unit operations.

Here we will demonstrate the three main components of MetPy's integration with xarray: 
* coordinates/coordinate reference systems
* units
* calculations

First, some general imports...


In [ ]:
%matplotlib inline
!python -m pip install 'metpy'
!pip install cartopy
!apt-get -qq install python-cartopy python3-cartopy
!pip uninstall -y shapely
!pip install shapely --no-binary shapely
!pip install cartopy


In [ ]:
from datetime import datetime, timedelta

import xarray as xr
import cartopy

import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.io import metar
from metpy.plots.declarative import (BarbPlot, ContourPlot, FilledContourPlot, MapPanel,
                                     PanelContainer, PlotObs)
from metpy.units import units

import numpy as np


...and opening some sample data to work with.



In [ ]:
# Open the netCDF file as a xarray Dataset
data = xr.open_dataset(get_test_data('irma_gfs_example.nc', False))

# View a summary of the Dataset
data

While xarray can handle a variety of n-dimensional data (essentially anything storable in a netCDF file), a common use case is working with gridded model output. Here we'll use an example subset of GFS data
from Hurricane Irma (September 5th, 2017) included in MetPy's test suite. 

Going back to the above object, this ``Dataset`` consists of:
 * *dimensions* 

 and their associated
 * *coordinates*
 
 which in turn make up the axes along which the 
 * *data variables*

are defined. The dataset also has a dictionary-like collection of 
* *attributes*. 

What happens if we look at just a single data variable?



In [ ]:
temperature = data['Temperature_isobaric']
temperature

This is a 
* ``DataArray``, 

which stores just a single data variable with its associated
coordinates and attributes. 

MetPy takes these kinds of ``DataArray`` objects as inputs.


## Coordinates and Coordinate Reference Systems

MetPy's first set of helpers comes with identifying *coordinate types*. In a given dataset,
coordinates can have a variety of different names and yet refer to the same type (such as
"isobaric1" and "isobaric3" both referring to vertical isobaric coordinates). Following
CF conventions, as well as using some fall-back regular expressions, MetPy can
systematically identify coordinates of the following types:

- time
- vertical
- latitude
- y
- longitude
- x

When identifying a single coordinate, it is best to use the property directly associated
with that type.



In [ ]:
temperature.metpy.time

You can use the ``.coordinates()``
method to yield a generator for coordinates when accessing multiple coordinate types simultaneously



In [ ]:
x, y = temperature.metpy.coordinates('x', 'y')

These coordinate type aliases can also be used in MetPy's wrapped ``.sel`` and ``.loc``
for indexing and selecting on ``DataArray``s. For example, to access 500 hPa heights at
1800Z,



In [ ]:
heights = data['Geopotential_height_isobaric'].metpy.sel(
    time='2017-09-05 18:00',
    vertical=50000.
)

Notice how we specified 50000 here without units...

Let's look at a better alternative.

## Units

Unit-aware calculations are a major part of the MetPy library, and unit support is a major
part of MetPy's xarray integration!

xarray data variables can store both unit-aware and unit-naive array types.

Xarray defaults to unit-naive array types, so we convert to a unit-aware type to use xarray operations while preserving unit correctness. MetPy
provides the ``.quantify()`` method for this (named since we are turning the data stored inside the xarray object into a Pint ``Quantity`` object)



In [ ]:
heights = heights.metpy.quantify()
heights

Notice how the units are now represented in the data itself, rather than as a text
attribute. Now, even if we perform some kind of xarray operation (such as taking the zonal
mean), the units are preserved:



In [ ]:
heights_mean = heights.mean('longitude')
heights_mean

However, this "quantification" is not without its consequences. By default, xarray loads its
data lazily to conserve memory usage. Unless your data is chunked into a Dask array (using
the ``chunks`` argument), this ``.quantify()`` method will load data into memory, which
could slow your script or even cause your process to run out of memory. And so, we recommend
subsetting your data before quantifying it.

Also, these Pint ``Quantity`` data objects are not properly handled by xarray when writing
to disk. And so, if you want to safely export your data, you will need to undo the
quantification with the ``.dequantify()`` method, which converts your data back to a
unit-naive array with the unit as a text attribute



In [ ]:
heights_mean_str_units = heights_mean.metpy.dequantify()
heights_mean_str_units

Other useful unit integration features include:

Unit-based selection/indexing:



In [ ]:
heights_at_45_north = data['Geopotential_height_isobaric'].metpy.sel(
    latitude=45 * units.degrees_north,
    vertical=300 * units.hPa
)
heights_at_45_north

Unit conversion:



In [ ]:
temperature_degc = temperature[0].metpy.convert_units('degC')
temperature_degc

To base unit conversion:



In [ ]:
temperature_degk = temperature_degc.metpy.convert_to_base_units()
temperature_degk

Unit conversion for coordinates:



In [ ]:
heights_on_hpa_levels = heights.metpy.convert_coordinate_units('isobaric3', 'hPa')
heights_on_hpa_levels['isobaric3']

Accessing just the underlying unit array:



In [ ]:
heights_unit_array = heights.metpy.unit_array
heights_unit_array

Accessing just the underlying units:



In [ ]:
height_units = heights.metpy.units
height_units


# MetPy Declarative Syntax Tutorial

The declarative syntax that is a part of the MetPy packaged is designed to aid in simple
data exploration and analysis needs by simplifying the plotting context from typical verbose
Python code. The complexity of data wrangling and plotting are hidden behind the simplified
syntax to allow a lower barrier to investigating your data.


In [ ]:
from metpy.io import metar
from metpy.plots.declarative import (BarbPlot, ContourPlot, FilledContourPlot, MapPanel,
                                     PanelContainer, PlotObs)
from datetime import datetime, timedelta
import cartopy

In [ ]:
# Open the netCDF file as a xarray Dataset and parse the full dataset
data = xr.open_dataset(get_test_data('GFS_test.nc', False)).metpy.parse_cf()

# View a summary of the Dataset
print(data)

## Set Datetime

Set the date/time of that you desire to plot


In [ ]:
plot_time = datetime(2010, 10, 26, 12)

## Subsetting Data

MetPy provides wrappers for the usual xarray indexing and selection routines that can handle
quantities with units. For DataArrays, MetPy also allows using the coordinate axis types
mentioned above as aliases for the coordinates. And so, if we wanted data to be just over
the U.S. for plotting purposes

In [ ]:
ds = data.metpy.sel(lat=slice(70, 10), lon=slice(360 - 150, 360 - 55))

## Calculations

MetPY calculation functions accept Xarray DataArray's as input and output a DataArray that can be easily added to an existing Dataset.

As an example, we calculate wind speed from the wind components and add it as a new variable
to our Dataset:



In [ ]:
ds['wind_speed'] = mpcalc.wind_speed(ds['u-component_of_wind_isobaric'],
                                     ds['v-component_of_wind_isobaric'])

## Plotting

With that minimal preparation, we are now ready to use the simplified plotting syntax to be
able to plot our data and analyze the meteorological situation.

General Structure

1. Set contour attributes

2. Set map characteristics and collect contours

3. Collect panels and plot

4. Show (or save) the results

Valid Plotting Types for Gridded Data:

- ``ContourPlot()``

- ``FilledContourPlot()``

- ``ImagePlot()``

- ``BarbPlot()``

Let's plot a 300-hPa map with color-filled wind speed, which we calculated and added to
our Dataset above, and geopotential heights over the CONUS.



We'll start by setting attributes for contours of Geopotential Heights at 300 hPa.
We need to set at least the data, field, level, and time attributes. We'll set a few others
to have greater control over hour the data is plotted.



In [ ]:
# Set attributes for contours of Geopotential Heights at 300 hPa
cntr2 = ContourPlot()
cntr2.data = ds
cntr2.field = 'Geopotential_height_isobaric'
cntr2.level = 300 * units.hPa
cntr2.time = plot_time
cntr2.contours = list(range(0, 10000, 120))
cntr2.linecolor = 'black'
cntr2.linestyle = 'solid'
cntr2.clabels = True

Now we'll set the attributes for plotting color-filled contours of wind speed at 300 hPa.
Again, the attributes that must be set include data, field, level, and time. We'll also set
a colormap and colorbar to be purposeful for wind speed. Additionally, we'll set the
attribute to change the units from m/s to knots, which is the common plotting units for
wind speed.

In [ ]:
# Set attributes for plotting color-filled contours of wind speed at 300 hPa
cfill = FilledContourPlot()
cfill.data = ds
cfill.field = 'wind_speed'
cfill.level = 300 * units.hPa
cfill.time = plot_time
cfill.contours = list(range(10, 201, 20))
cfill.colormap = 'BuPu'
cfill.colorbar = 'horizontal'
cfill.plot_units = 'knot'

Once we have our contours (and any colorfill plots) set up, we will want to define the map
panel that we'll plot the data on. This is the place where we can set the view extent,
projection of our plot, add map lines like coastlines and states, set a plot title.
One of the key elements is to add the data to the map panel as a list with the plots
attribute.



In [ ]:
# Set the attributes for the map and add our data to the map
panel = MapPanel()
panel.area = [-125, -74, 20, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'{cfill.level.m}-hPa Heights and Wind Speed at {plot_time}'
panel.plots = [cfill, cntr2]

Finally we'll collect all the panels to plot on the figure, set the size of the figure,
and ultimately show or save the figure.


In [ ]:
# Set the attributes for the panel and put the panel in the figure
pc = PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]

In [ ]:
# Show the image
pc.show()

--------------------------------------
# **BONUS ROUND**: Open Global Glacier Model + Salem

<center><img src="
https://oggm.org/tutorials/stable/_static/logo.png" alt="Drawing" style="width: 450px;"/></center>


<center><h1 style=font-size:500%> + </h1></center>


<center><img src="https://www.insidehook.com/wp-content/uploads/2020/10/salem_ma.jpg?fit=3563%2C2375" alt="Drawing" style="width: 100px;"/> </center>



In [ ]:
!python -m pip install 'geopandas'
!python -m pip install 'salem'
!python -m pip install 'oggm'


In [ ]:
## Libs
import matplotlib.pyplot as plt
import salem
import geopandas as gpd

# OGGM
import oggm.cfg as cfg
from oggm import utils, workflow, tasks, graphics

In [ ]:
# Initialize OGGM and set up the default run parameters
cfg.initialize(logging_level='WARNING')
cfg.PARAMS['use_multiprocessing'] = False
# Local working directory (where OGGM will write its output)
cfg.PATHS['working_dir'] = utils.gettempdir('OGGM_Toy_Thickness_Model')
# We use the directories with the shop data in it: "W5E5_w_data"
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/elev_bands/W5E5_w_data/'

# Pick our glacier
gdir = workflow.init_glacier_directories(['RGI60-01.16195'], 
                                          from_prepro_level=3, 
                                          prepro_base_url=base_url,
                                          prepro_border=10)[0]
gdir                

We are going to use the South Glacier example taken from the [ITMIX experiment](https://www.the-cryosphere.net/11/949/2017/). It is a small (5.6 km2) glacier in Alaska.

In [ ]:
with xr.open_dataset(gdir.get_filepath('gridded_data')) as ds:
    ds = ds.load()
# List all variables
ds

In [ ]:
import salem
# import matplotlib.pyplot as plt
smap = ds.salem.get_map(countries=False)
smap.set_shapefile(gdir.read_shapefile('outlines'))
smap.set_topography(ds.topo.data);

# ITSLive velocity data 

In [ ]:
# get the velocity data
u = ds.itslive_vx.where(ds.glacier_mask)
v = ds.itslive_vy.where(ds.glacier_mask)
ws = ds.itslive_v.where(ds.glacier_mask)

The `.where(ds.glacier_mask)` command will remove the data outside of the glacier outline.

In [ ]:
# get the axes ready
f, ax = plt.subplots(figsize=(9, 9))

# Quiver only every N grid point
us = u[1::3, 1::3]
vs = v[1::3, 1::3]

smap.set_data(ws)
smap.set_cmap('Blues')
smap.plot(ax=ax)
smap.append_colorbar(ax=ax, label='ice velocity (m yr$^{-1}$)')

# transform their coordinates to the map reference system and plot the arrows
xx, yy = smap.grid.transform(us.x.values, us.y.values, crs=gdir.grid.proj)
xx, yy = np.meshgrid(xx, yy)
qu = ax.quiver(xx, yy, us.values, vs.values)
qk = ax.quiverkey(qu, 0.82, 0.97, 10, '10 m yr$^{-1}$',
                  labelpos='E', coordinates='axes')
ax.set_title('ITS-LIVE velocity');

## Millan 2022 velocity data 

In [ ]:
# get the velocity data
u = ds.millan_vx.where(ds.glacier_mask)
v = ds.millan_vy.where(ds.glacier_mask)
ws = ds.millan_v.where(ds.glacier_mask)

In [ ]:
# get the axes ready
f, ax = plt.subplots(figsize=(9, 9))

# Quiver only every N grid point
us = u[1::3, 1::3]
vs = v[1::3, 1::3]

smap.set_data(ws)
smap.set_cmap('Blues')
smap.plot(ax=ax)
smap.append_colorbar(ax=ax, label='ice velocity (m yr$^{-1}$)')

# transform their coordinates to the map reference system and plot the arrows
xx, yy = smap.grid.transform(us.x.values, us.y.values, crs=gdir.grid.proj)
xx, yy = np.meshgrid(xx, yy)
qu = ax.quiver(xx, yy, us.values, vs.values)
qk = ax.quiverkey(qu, 0.82, 0.97, 10, '10 m yr$^{-1}$',
                  labelpos='E', coordinates='axes')
ax.set_title('Millan 2022 velocity');

## Thickness data from Farinotti 2019 and Millan 2022 

In [ ]:
# get the axes ready
f, ax = plt.subplots(figsize=(9, 9))
smap.set_cmap('viridis')
smap.set_data(ds.consensus_ice_thickness)
smap.plot(ax=ax)
smap.append_colorbar(ax=ax, label='ice thickness (m)')
ax.set_title('Farinotti 2019 thickness');

In [ ]:
# get the axes ready
f, ax = plt.subplots(figsize=(9, 9))
smap.set_cmap('viridis')
smap.set_data(ds.millan_ice_thickness.where(ds.glacier_mask))
smap.plot(ax=ax)
smap.append_colorbar(ax=ax, label='ice thickness (m)')
ax.set_title('Millan 2022 thickness');

Let's also add some attributes that OGGM can compute for us:


In [ ]:
# Tested tasks
task_list = [
    tasks.gridded_attributes,
    tasks.gridded_mb_attributes,
]
for task in task_list:
    workflow.execute_entity_task(task, gdir)

Let's open the gridded data file again with xarray:

In [ ]:
with xr.open_dataset(gdir.get_filepath('gridded_data')) as ds:
    ds = ds.load()
# List all variables
ds

The file contains several new variables with their description. For example:

In [ ]:
ds.oggm_mb_above_z

Let's plot a few of them (we show how to plot them with xarray and with oggm:

In [ ]:
ds.slope.plot();
plt.axis('equal');

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
graphics.plot_raster(gdir, var_name='aspect', cmap='twilight', ax=ax1)
graphics.plot_raster(gdir, var_name='oggm_mb_above_z', ax=ax2)

<div class="alert alert-info">
    <b>
        In a few lines of code, we have used OGGM to generate or deliver a bunch of data for this glaciers. A similar workflow can be used on ALL of them!
    </b>
</div>